Only keep (not ablate) the query activations (query vector, the output of query weights times inputs) of certain positions. Keep all the key activations; the query positions that were kept will automatically attend to relevant key positions by matrix multiplication.

# Setup
(No need to change anything)

## import libs

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-t0x3r5hh
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-t0x3r5hh
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit fa287750606075574df2c538058e67d648e2f952
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.5 MB/s eta 0:00:00
   

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [4]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [5]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [6]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [7]:
model = HookedTransformer.from_pretrained(
    "gpt2-medium",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Loaded pretrained model gpt2-medium into HookedTransformer


## Import functions from repo

In [8]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

Cloning into 'ARENA_2.0'...
remote: Enumerating objects: 9106, done.
remote: Counting objects: 100% (1818/1818), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 9106 (delta 1611), reused 1607 (delta 1527), pack-reused 7288
Receiving objects: 100% (9106/9106), 155.60 MiB | 15.33 MiB/s, done.
Resolving deltas: 100% (5506/5506), done.


In [9]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [10]:
import ioi_circuit_extraction as ioi_circuit_extraction

# Generate dataset with multiple prompts

In [22]:
class Dataset:
    def __init__(self, prompts, pos_dict, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["corr"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["incorr"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'corr' and key != 'incorr')]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                # if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                #     target_token = prompt[targ]
                # else:
                #     target_token = "Ġ" + prompt[targ]
                # target_index = tokens.index(target_token)
                target_index = pos_dict[targ]
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

In [23]:
pos_dict = {
    'S1': 0,
    'S2': 1,
    'S3': 2,
    'S4': 3,
    'S5': 4,
    'S6': 5,
}

In [24]:
def generate_prompts_list(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+2),
            'S3': str(i+4),
            'S4': str(i+6),
            'S5': str(i+8),
            'S6': str(i+10),
            'corr': str(i+12),
            'incorr': str(i+10),
            'text': f"{i} {i+2} {i+4} {i+6} {i+8} {i+10}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(1, 101)
dataset = Dataset(prompts_list, pos_dict, model.tokenizer, S1_is_first=True)

In [25]:
import random

def generate_prompts_list_corr(x ,y):
    prompts_list = []
    for i in range(x, y):
        r1 = random.randint(1, 100)
        r2 = random.randint(1, 100)
        r3 = random.randint(1, 100)
        r4 = random.randint(1, 100)
        r5 = random.randint(1, 100)
        r6 = random.randint(1, 100)
        prompt_dict = {
            'S1': str(r1),
            'S2': str(r2),
            'S3': str(r3),
            'S4': str(r4),
            'S5': str(r5),
            'S6': str(r6),
            'corr': str(r1),
            'incorr': str(i+6),
            'text': f"{r1} {r2} {r3} {r4} {r5} {r6}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list_2 = generate_prompts_list_corr(1, 101)
dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer, S1_is_first=True)
prompts_list_2

[{'S1': '52',
  'S2': '11',
  'S3': '37',
  'S4': '78',
  'S5': '7',
  'S6': '1',
  'corr': '52',
  'incorr': '7',
  'text': '52 11 37 78 7 1'},
 {'S1': '62',
  'S2': '65',
  'S3': '73',
  'S4': '90',
  'S5': '53',
  'S6': '75',
  'corr': '62',
  'incorr': '8',
  'text': '62 65 73 90 53 75'},
 {'S1': '25',
  'S2': '2',
  'S3': '38',
  'S4': '23',
  'S5': '92',
  'S6': '37',
  'corr': '25',
  'incorr': '9',
  'text': '25 2 38 23 92 37'},
 {'S1': '4',
  'S2': '49',
  'S3': '81',
  'S4': '93',
  'S5': '5',
  'S6': '14',
  'corr': '4',
  'incorr': '10',
  'text': '4 49 81 93 5 14'},
 {'S1': '14',
  'S2': '85',
  'S3': '91',
  'S4': '17',
  'S5': '45',
  'S6': '7',
  'corr': '14',
  'incorr': '11',
  'text': '14 85 91 17 45 7'},
 {'S1': '79',
  'S2': '67',
  'S3': '14',
  'S4': '39',
  'S5': '35',
  'S6': '69',
  'corr': '79',
  'incorr': '12',
  'text': '79 67 14 39 35 69'},
 {'S1': '4',
  'S2': '66',
  'S3': '74',
  'S4': '21',
  'S5': '25',
  'S6': '4',
  'corr': '4',
  'incorr': '13',
 

Logit diff is correct - incorr token. Here, correct is S5, and incorr is S4.

Because of this, it's possible to have logit diffs HIGHER than the "full circuit" because the correct token will still be at first place, but the logit scores assigned will just be bigger (perhaps incorrect is scored even lower in the non-full circuit with a higher logit diff score)?

# Ablation Expm Functions

In [26]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

We can also prevent redundant computation of the full circuit score by storing it and just passing it in to the function.

In [27]:
ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)
orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

In [28]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)
logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

tensor(1.5984, device='cuda:0')

In [29]:
def mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=True):
    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

    # ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

    model_abl = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
    ioi_logits_minimal = model_abl(dataset.toks)

    # orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

    new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
    if print_output:
        # print(f"Average logit difference (IOI dataset, using entire model): {orig_score:.4f}")
        # print(f"Average logit difference (IOI dataset, only using circuit): {new_score:.4f}")
        print(f"Average logit difference (circuit / full) %: {100 * new_score / orig_score:.4f}")
    # return new_score
    return 100 * new_score / orig_score

In [30]:
lst = [(0, 0)]

CIRCUIT = {
    "number mover": lst,
    "number mover 5": lst,
    "number mover 4": lst,
    "number mover 3": lst,
    "number mover 2": lst,
    "number mover 1": lst,
}

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    "number mover 5": "S5",
    "number mover 4": "S4",
    "number mover 3": "S3",
    "number mover 2": "S2",
    "number mover 1": "S1",
}

model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
model_abl = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
ioi_logits_minimal = model_abl(dataset.toks)
logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)

tensor(-0.1129, device='cuda:0')

# Ablate the model and compare with original

In [31]:
lst = [(layer, head) for layer in range(24) for head in range(16)]

CIRCUIT = {
    "number mover": lst,
    "number mover 5": lst,
    "number mover 4": lst,
    "number mover 3": lst,
    "number mover 2": lst,
    "number mover 1": lst,
}

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    "number mover 5": "S5",
    "number mover 4": "S4",
    "number mover 3": "S3",
    "number mover 2": "S2",
    "number mover 1": "S1",
}

mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=False).item()

100.0

In [32]:
def circuit_from_headsList(headsList):
    CIRCUIT = {
    "number mover": headsList,
    "number mover 5": headsList,
    "number mover 4": headsList,
    "number mover 3": headsList,
    "number mover 2": headsList,
    "number mover 1": headsList,
    }

    SEQ_POS_TO_KEEP = {
        "number mover": "end",
        "number mover 5": "S5",
        "number mover 4": "S4",
        "number mover 3": "S3",
        "number mover 2": "S2",
        "number mover 1": "S1",
    }
    return CIRCUIT, SEQ_POS_TO_KEEP

In [33]:
CIRCUIT, SEQ_POS_TO_KEEP = circuit_from_headsList([(0,0)])
mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=False).item()

-7.061283111572266

In [ ]:
# Start with full circuit
curr_circuit = [(layer, head) for layer in range(24) for head in range(16)]
threshold = 3  # This is T, a %. if performance is less than T%, allow its removal

for layer in range(23, -1, -1):  # go thru all heads in a layer first
    for head in range(16):
        # Copying the curr_circuit so we can iterate over one and modify the other
        copy_circuit = curr_circuit.copy()

        # Temporarily removing the current tuple from the copied circuit
        copy_circuit.remove((layer, head))

        CIRCUIT, SEQ_POS_TO_KEEP = circuit_from_headsList(copy_circuit)
        new_score = mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=False).item()

        # print((layer,head), new_score)
        # If the result is less than the threshold, remove the tuple from the original list
        if (100 - new_score) < threshold:
            curr_circuit.remove((layer, head))

            print("Removed:", (layer, head))
            print(new_score)
            print("\n")

(23, 0) 100.32792663574219
Removed: (23, 0)
100.32792663574219


(23, 1) 100.25212860107422
Removed: (23, 1)
100.25212860107422


(23, 2) 100.70158386230469
Removed: (23, 2)
100.70158386230469


(23, 3) 99.92982482910156
Removed: (23, 3)
99.92982482910156


(23, 4) 100.03290557861328
Removed: (23, 4)
100.03290557861328


(23, 5) 100.40460205078125
Removed: (23, 5)
100.40460205078125


(23, 6) 100.61436462402344
Removed: (23, 6)
100.61436462402344


(23, 7) 100.69398498535156
Removed: (23, 7)
100.69398498535156


(23, 8) 99.70844268798828
Removed: (23, 8)
99.70844268798828


(23, 9) 99.7198257446289
Removed: (23, 9)
99.7198257446289


(23, 10) 99.79279327392578
Removed: (23, 10)
99.79279327392578


(23, 11) 100.00129699707031
Removed: (23, 11)
100.00129699707031


(23, 12) 100.12457275390625
Removed: (23, 12)
100.12457275390625


(23, 13) 100.92903137207031
Removed: (23, 13)
100.92903137207031


(23, 14) 100.98147583007812
Removed: (23, 14)
100.98147583007812


(23, 15) 100.804916381835

In [ ]:
curr_circuit

In [ ]:
len(curr_circuit)

384 total